# Step 4: Retrosynthetic Planning

This tutorial demonstrates how retrosynthetic planning can be performed for target molecules in ``SynPlanner``. For retrosynthetic planning, the retrosynthetic models trained with the previous tutorials can be used. Alternatively,  the pre-trained retrosynthetic models downloaded from ``SynPlanner`` can be used.

## Basic recommendations

**1. The "Evaluation first" search strategy is not compatible with the rollout evaluation**

In ``SynPlanner``, there are two main search strategies implemented - the "Expansion first" and "Evaluation first" strategy. These are the strategies for navigating the search tree. "Expansion first" prioritizes the expansion of new nodes and assigns to each new node the default value. "Evaluation first" prioritizes the evaluation of each new node first. Notice, that the usage of the "Evaluation first" strategy with the current implementation of the rollout function in ``SynPlanner`` is not reasonable in terms of the total search time because of the time-consuming execution of the rollout function. Also, the current implementation of the rollout function may mislead the search in the case of the "Evaluation first" strategy due to the limited explanation of the tree in rollout simulations. Therefore, the current recommendation is to use the "Evaluation first" strategy with value network evaluation only.

**2. Try more search iterations (and longer search time) with complex molecules, a limited set of reaction rules, or building blocks**

Some target molecules (usually more complex and bigger molecules) require longer tree searches to be successfully solved and may require longer retrosynthesis routes. The same is true, if there is only a mall set of reaction rules or building blocks, which again requires for longer analysis to find the proper combination of reaction rules leading to the limited amount of building blocks. In these cases, the increase in the number of search iterations may help to find the successful retrosynthesis route for the given molecule.

**3. Currently retrosynthesis planning results in many similar retrosynthetic routes**

Currently, there is no implementation of the retrosynthesis routes clustering in ``SynPlanner``. It may lead to the generation of many similar retrosynthesis routes for the target molecule.

## 1. Set up input and output data locations

The ``SynPlanner`` input data will be downloaded from the ``HuggingFace`` repository to the specified directory.

For the retrosynthetic planning the following data and files are needed:
    
| Data / Files | Description |
| --- | --- |
| Reaction rules  | Extracted reaction rules for precursors dissection in retrosynthetic planning |
| Policy network  | Trained ranking or filtering policy network for node expansion in tree search |
| Value network   | Trained value neural network for node evaluation in tree search (optional, the default evaluation method is rollout) |
| Building blocks | Set of building block molecules, which are used as terminal materials in the retrosynthetic route planning |

In [1]:
from pathlib import Path
from synplan.utils.loading import download_all_data

# download SynPlanner data
data_folder = Path("synplan_data").resolve()
download_all_data(save_to=data_folder)
ranking_policy_network = data_folder.joinpath("uspto/weights/ranking_policy_network.ckpt").resolve(strict=True)
reaction_rules_path = data_folder.joinpath("uspto/uspto_reaction_rules.pickle")
# use your custom building blocks if needed
building_blocks_path = data_folder.joinpath("building_blocks/building_blocks_em_sa_ln.smi")

# planning results folder
results_folder = Path("tutorial_results").resolve()
results_folder.mkdir(exist_ok=True)

Fetching 25 files:   0%|          | 0/25 [00:00<?, ?it/s]

## 2. Tree search configuration

The search tree in SynPlanner is represented by `Tree` class, which requires:

* loaded building blocks provided as CGRtools `MolecularContainer` object
* loaded reaction rules provided as CGRtools `Reactor` objects
* loaded expansion function represented as trained policy network.
* tree configuration of SynPlanner `TreeConfig` object
* target molecule provided as CGRtools `MolecularContainer` object

### Loading building blocks

Building blocks can be loaded with the `load_building_blocks` function. They can be loaded from a `.pickle` file containing already standardized molecules as `MolecularContainer` or from a `.smi` file containing SMILES. 

<div class="alert alert alert-warning">
<b>Warning</b> 
    
The first loading of building blocks can be long, especially if they loaded from SMILES file.
</div>

In [2]:
from synplan.utils.loading import load_building_blocks

building_blocks = load_building_blocks(building_blocks_path, standardize=True)

### Loading reaction rules

Reaction rules can be loaded with `load_reaction_rules` function that will automatically convert them to `Reactor` object of CGRtools. Currently, they are loaded from `.pickle` file, that was generated during reaction rule extraction. The loading from SMARTS would be added in the next versions. 

For more information, please visit rules extraction tutorial.

In [3]:
from synplan.utils.loading import load_reaction_rules

reaction_rules = load_reaction_rules(reaction_rules_path)

### Loading policy network

Policy function can be loaded directly from the weights file ends with `.ckpt`, as it includes all configuration for the policy network used for its training 

<div class="alert alert-info">
<b>Note</b>   

    
The `PolicyNetworkConfig` parameters are ignored when the `.ckpt` file is provided, as it already contains all parameter values
</div>

For more info, please visit ranking policy training tutorial. 

In [4]:
from synplan.utils.loading import load_policy_function

policy_function = load_policy_function(weights_path=ranking_policy_network)

Lightning automatically upgraded your loaded checkpoint from v1.9.5 to v2.5.1.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint synplan_data/uspto/weights/ranking_policy_network.ckpt`


### Search confuguration

The next step is to configure the Monte-Carlo Tree Search. We do this using the `TreeConfig` class in ``SynPlanner``. This class allows for the specification of various parameters and settings for the MCTS.

More details about MCTS configuration in ``SynPlanner`` can be found in <a href="https://synplanner.readthedocs.io/en/latest/configuration/planning.html">official documentation</a>.

In [5]:
from synplan.utils.config import TreeConfig

tree_config = TreeConfig(
    search_strategy="expansion_first",
    max_iterations=300,
    max_time=120,
    max_depth=9,
    min_mol_size=1,
    init_node_value=0.5,
    ucb_type="uct",
    c_ucb=0.1,
)

### Evaluation strategy configuration

The evaluation strategy determines how nodes are scored during tree search. We create an evaluation configuration that contains all necessary dependencies.


In [6]:
from synplan.utils.config import RolloutEvaluationConfig
from synplan.utils.loading import load_evaluation_function

# Create evaluation configuration
eval_config = RolloutEvaluationConfig(
    policy_network=policy_function,
    reaction_rules=reaction_rules,
    building_blocks=building_blocks,
    min_mol_size=tree_config.min_mol_size,
    max_depth=tree_config.max_depth,
)

# Create evaluator from config
evaluation_function = load_evaluation_function(eval_config)

### Loading target molecule

The target molecule need to be loaded as `MolecularContainer` and **must be standardized**. The easiest way is to use `smiles` function availible in CGRtools. 

For the search with multiple molecules, please visit advanced retrosynthetic planning tutorial.

In [7]:
from synplan.chem.utils import mol_from_smiles

# let's take capivasertib used as anti-cancer medication for the treatment 
# of breast cancer and approved by FDA in 2023
example_smiles = "NC1(C(=O)N[C@@H](CCO)c2ccc(Cl)cc2)CCN(c2nc[nH]c3nccc2-3)CC1"

target_molecule = mol_from_smiles(example_smiles, clean2d=True, standardize=True, clean_stereo=True)

### Initialising tree

Next, we initialise the `Tree` object, providing target molecule and loaded reaction rules, building blocks, expansion function, and evaluation strategy.

In [8]:
from synplan.mcts.tree import Tree

tree = Tree(
    target=target_molecule,
    config=tree_config,
    reaction_rules=reaction_rules,
    building_blocks=building_blocks,
    expansion_function=policy_function,
    evaluation_function=evaluation_function,
)

## 3. Running retrosynthetic planning

The `Tree` object is iterable, like `List` in Python. Each iteration represents one MCTS iteration, after which 2 values are returned:

* `solved` is boolean, if `True` then a successful route was found during the current iteration.
* `node_id` is a list of indices of leaf nodes generated during the current iteration.

Once search is finished, the tree will print some stats:

In [9]:
tree_solved = False
for solved, node_id in tree:
    if solved:
        tree_solved = True
tree

  0%|          | 0/300 [00:00<?, ?it/s]

Tree for: c1cc(ccc1Cl)C(CCO)NC(C2(CCN(CC2)c3c4cc[nH]c4ncn3)N)=O
Time: 24.1 seconds
Number of nodes: 4314
Number of iterations: 300
Number of visited nodes: 300
Number of found routes: 153

### Visualizing predicted retrosynthetic routes

After the tree search is complete, we can visualize the found retrosynthesis paths. The visualization uses the `get_route_svg` function from SynPlanner visualization interface.

In [10]:
from IPython.display import SVG, display
from synplan.utils.visualisation import get_route_svg

for n, node_id in enumerate(tree.winning_nodes):
    print(
        f"-------- Path starts from node #{node_id} with total route score {tree.route_score(node_id)} --------"
    )
    display(SVG(get_route_svg(tree, node_id)))
    if n == 3:
        break

-------- Path starts from node #85 with total route score 0.11169304955365641 --------


-------- Path starts from node #86 with total route score 0.11169304955365641 --------


-------- Path starts from node #88 with total route score 0.11169304955365641 --------


-------- Path starts from node #157 with total route score 0.1494125627057903 --------
